In [1]:
clean_up = True
%run stdPackages.ipynb
from pyDatabases import gpyDB, gpyDB_wheels
rc_pd = gpyDB.adj.rc_pd

The file_gams_py_gdb0.gdx is still active and was not deleted.
The file_gams_py_gdb1.gdx is still active and was not deleted.


The notebook shows how we can load IO data in a smart way that automatically defines mappings. The IO data automatically include a number of different sheets; this split helps us to identify some of the subsets.
* ```p```: Includes all domestic production sectors. We may want to split this up further, e.g. if some sector has to be paid specific attention.
* ```f```: foreign sector/trade. 
* ```i```: Investment.
* ```h```: households.
* ```g```: government.

In [2]:
name = 'A'
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(1,11)})
db = gpyDB.GpyDB(**{'name': f"IO_{name}", 'alias': pd.MultiIndex.from_tuples([('n','nn'),('n','nnn')])})
wv = gpyDB_wheels.read.simpleLoad(f"{d['data']}\\A_IOv.xlsx")
wp = gpyDB_wheels.read.simpleLoad(f"{d['data']}\\A_IOp.xlsx")
dur= gpyDB_wheels.read.SeriesDB_from_wb(f"{d['data']}\\A_dur.xlsx", {'variables': ['pV'], 'maps': ['pM']})

*Load in all modules and define sectors + outputs if ```vS``` is in the data:*

In [3]:
ms = ['p','f','i','h','g']
dbs_v = {m: gpyDB_wheels.read.SeriesDB_from_wb(wv, {'variables': m}) for m in ms}
dbs_p = {m: gpyDB_wheels.read.SeriesDB_from_wb(wp, {'variables': m}) for m in ('n','i')}
for m in ms:
    if 'vS' in dbs_v[m].database:
        dbs_v[m][f"s_{m}"] = dbs_v[m]['vS'].index.levels[0]
        dbs_v[m][f"nOut_{m}"] = dbs_v[m]['vS'].index.levels[-1]
dbs_v['f']['nOut_f'] = dbs_v['f'].get('vD').index.levels[-1].difference(dbs_v['p'].get('nOut_p'))
dbs_v['f']['s_f'] = dbs_v['f'].get('vD').index.levels[0].difference(dbs_v['p'].get('s_p'))
dbs_v['p']['dur_p'] = dur.get('qD').index
dbs_v['p']['inv_p'] = dbs_v['i']['nOut_i'].vals

Merge databases as IO data:

In [4]:
[gpyDB_wheels.robust.robust_merge_dbs(db, db_i, priority='second') for db_i in dbs_v.values()];
[gpyDB_wheels.robust.robust_merge_dbs(db, db_i, priority='second') for db_i in dbs_p.values()];
[gpyDB_wheels.robust.robust_merge_dbs(db, db_i, priority='second') for db_i in [dur]];

Compute prices on durables from the steady state expression:
$$\begin{align}
    p_K = p_I\left(\dfrac{R}{1+\pi}-(1-\delta)\right),
\end{align}$$
where $K$ is the durable, $I$ the corresponding investment variable, $R$ is the long run interest rate factor, $\pi$ is the long run inflation rate, and $\delta$ is the depreciation rate.

In [5]:
pDur = pyDatabases.adjMultiIndex.applyMult( db.get('p').rename_axis(index={'n':'nn'}), db.get('dur2inv')).droplevel('nn') * (glob.db['R_LR'].vals / (1+glob.db['infl_LR'].vals) +db.get('rDepr')-1)
db['qD'] = (db.get('vD') / db.get('p')).fillna(dur.get('qD'))
db['qS'] = db['vS'].vals / db.get('p')

*TEMPORARY FIX, BECAUSE WE DO NOT HAVE A GOVERNMENT SECTOR YET: Set the pS/pD prices from the equilibrium prices*

In [6]:
db['pD'] = pDur.combine_first(pyDatabases.adjMultiIndex.bc(db.get('p'), db['qD']))
db['pS'] = pyDatabases.adjMultiIndex.bc(db.get('p'), db['qS'].vals)

Broadcast all variables using the time index (repeat variables for all $t$):

In [7]:
def addT(symbol, t):
    return pyDatabases.adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db.__setitem__(k, addT(db.get(k), glob.db['t'].vals)) for k in db.getTypes(['variable','scalar_variable'])];

To do: 
* Currently, we use the default options for the ```global``` settings. This includes long run growth rate, inflation rate, interest rate etc. This should be modified somewhere (potentially just in the JupyterLab code).
* Taxes are not implemented yet; the implication is that we do not have a good way of implementing prices after taxes (pD). Thus, currently, we simply use the equilibrium prices as basis for this.

In [8]:
pyDatabases.gpyDB_wheels.aggregateDB.readSets(db)

In [9]:
db.export(repo = d['data'])
with open(f"{d['data']}\\glob_{name}", "wb") as file:
    pickle.dump(glob,file)